# # To-Do:
1. Construct baseline model + saving specification 
2. Calculate epoch specifications 
3. Figure out how to tie graph to GPU device 
4. start baseline training 

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from trackml.dataset import load_event, load_dataset
from trackml.randomize import shuffle_hits
from trackml.score import score_event
import os
import math 
import time
%run utils.ipynb

In [2]:
N_STEPS = 10
N_EPOCHS = 3
INPUTS_DIM = 5
OUTPUTS_DIM = 3
LSTM_NEURONS = 400
LEARN_RATE = 0.0005 
BATCH_SIZE = 50
n_iterations = 1000000000000000000000
n_epochs = 2

In [ ]:

global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name = 'global_step')
#with tf.device('/device:GPU:0'):

X = tf.placeholder(tf.float32, [None, N_STEPS, INPUTS_DIM], name = 'input_ph' )
Y = tf.placeholder(tf.float32, [None, N_STEPS, OUTPUTS_DIM], name = 'label_ph' )
#constructing model

lstm = tf.contrib.rnn.LSTMCell(num_units = LSTM_NEURONS, use_peepholes = True)
#adding FC layer
lstm_cell = tf.contrib.rnn.OutputProjectionWrapper(lstm, output_size = OUTPUTS_DIM)
rnn_outputs, hidden_states = tf.nn.dynamic_rnn(lstm_cell, X, dtype = tf.float32)

#prediction = tf.Variable(rnn_outputs([BATCH_SIZE, N_STEPS, OUTPUTS_DIM]), name = 'prediction')
#print_op might not be necessary

loss_diff = tf.reduce_sum(tf.square(tf.subtract(rnn_outputs, Y)), 2)
mask = tf.sign(tf.reduce_max(tf.abs(Y), 2))
mask_loss = mask*loss_diff
batch_loss = tf.reduce_mean(mask_loss)

train = tf.train.AdamOptimizer(learning_rate=LEARN_RATE)
train_op = train.minimize(batch_loss, global_step=global_step)

print_op = tf.Print(rnn_outputs, [rnn_outputs], name = 'print_op')
init=tf.global_variables_initializer()
    #data_itr = next_batch(N_STEPS, BATCH_SIZE, INPUTS_DIM)






with tf.name_scope('summaries'):
    tf.summary.scalar('loss', batch_loss)
    tf.summary.histogram('histogram_loss', batch_loss)
    summary_op = tf.summary.merge_all()

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as baseline_idealv4:
    init.run()
    initial_step = global_step.eval()
    saver = tf.train.Saver()
    writer = tf.summary.FileWriter('./logs/baseline-idealv4/', baseline_idealv4.graph)
    ckpt = tf.train.get_checkpoint_state(os.path.dirname('./checkpoints/baseline-idealv4/'))
    # if that checkpoint exists, restore from checkpoint
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(baseline_idealv4, ckpt.model_checkpoint_path)
    

    data_itr = next_batch(N_STEPS, BATCH_SIZE, INPUTS_DIM, simple = True, full_tracks = True, ideal = True, full_data = True)
    epochs = 0
    start = time.time()
    for iteration in range(n_iterations):
        
        try:
            x_batch, y_batch, _, __ = next(data_itr)

            ___, loss_batch, summary, test_print = baseline_idealv4.run([train_op, batch_loss, summary_op, print_op], feed_dict = {X:x_batch, Y:y_batch})
        
            writer.add_summary(summary, global_step = global_step.eval(session=baseline_idealv4))
            #print('input:')
            #print(x_batch)
            #print('pred:')
            #print(test_print)
            if iteration %100 ==0:
                mse = batch_loss.eval(feed_dict={X:x_batch, Y:y_batch})
                print(iteration, "\tMSE:", mse)
                saver.save(baseline_idealv4, './checkpoints/baseline-idealv4/', global_step = global_step)
        except StopIteration as e:
            epochs +=1
            if epocs == n_epochs:
                print('finished all epochs')
                baseline_idealv4.close()
                break
            else:
                data_itr = next_batch(N_STEPS, BATCH_SIZE, INPUTS_DIM, simple = True, full_tracks = True, ideal = True, full_data = True)
                end = time.time()
                print('elasped time for 1st epoch: '+str(end-start))
                pass
                    
                

Instructions for updating:
Use the retry module or similar alternatives.
INFO:tensorflow:Restoring parameters from ./checkpoints/baseline-idealv4/-19302
0 	MSE: 54.158375
100 	MSE: 57.281895
200 	MSE: 94.15044
300 	MSE: 97.87722
400 	MSE: 77.16811
500 	MSE: 48.507458
600 	MSE: 75.20193
700 	MSE: 33.33127
800 	MSE: 29.551706
900 	MSE: 48.722046
1000 	MSE: 38.667408
1100 	MSE: 57.85515
1200 	MSE: 51.395897
1300 	MSE: 30.696016
1400 	MSE: 510.6043
1500 	MSE: 39.591095
1600 	MSE: 32.449646
1700 	MSE: 41.39968
1800 	MSE: 36.389782
1900 	MSE: 40.102757
2000 	MSE: 34.95057
2100 	MSE: 41.551907
2200 	MSE: 30.315434
2300 	MSE: 25.112068
2400 	MSE: 52.183426
2500 	MSE: 32.96162
2600 	MSE: 20.011097
2700 	MSE: 48.377827
2800 	MSE: 75.404594
2900 	MSE: 71.95022
3000 	MSE: 29.8967
3100 	MSE: 31.342691
3200 	MSE: 23.355536
3300 	MSE: 38.792133
3400 	MSE: 48.060482
3500 	MSE: 60.873055
3600 	MSE: 25.671043
3700 	MSE: 47.16125
3800 	MSE: 84.69738
3900 	MSE: 132.8261
4000 	MSE: 33.807964
4100 	MSE: 26.